# ProyectoDSParteI_Hamburg

## Abstract

Este proyecto realiza un Análisis Exploratorio de Datos (EDA) sobre canciones populares utilizando el archivo **songs_normalize.csv**. El objetivo es comprender cómo se relacionan variables musicales cuantitativas con indicadores de desempeño, tomando como referencia la métrica de **popularidad** cuando esté disponible. El análisis se limita estrictamente a los contenidos vistos en el curso: carga de datos con Pandas, estadísticas descriptivas, tratamiento de datos ausentes, visualizaciones con Matplotlib/Seaborn (histogramas, boxplots, gráficos de dispersión, mapa de correlaciones) y detección de outliers mediante el criterio IQR. No se incluyen técnicas avanzadas ni bibliotecas fuera del programa.

El conjunto de datos contiene características típicas de audio tales como **energy**, **danceability**, **acousticness**, **valence**, **tempo/BPM**, **loudness**, **duration** y metadatos como **año de lanzamiento**. A partir de estas variables se construye una narrativa descriptiva para responder preguntas guía sobre patrones y asociaciones. La preparación se limita a la estandarización de nombres de columnas y a la comprobación de datos faltantes. Las visualizaciones se diseñan bajo los principios de buena comunicación: títulos, ejes y leyendas claras y un estilo coherente con Seaborn.

El trabajo propone hipótesis sobre la relación entre energía y bailabilidad con la popularidad, la influencia de la duración y del tempo, y posibles vínculos entre modo afectivo (valence) y recepción del público. Para contrastarlas se utilizan histogramas para entender distribuciones univariadas, boxplots para comparar grupos por **año** (en décadas) y gráficos multivariados que combinan color y tamaño para incorporar una tercera o cuarta variable. Finalmente, un **heatmap** de correlaciones resume la asociación lineal entre métricas, mientras que el IQR permite identificar observaciones atípicas que podrían sesgar análisis posteriores.

Los hallazgos permiten priorizar variables con mayor poder explicativo preliminar y ofrecen una base sólida para fases futuras del proyecto, como la selección de características y la construcción de modelos supervisados básicos. Este documento es autónomo, reproducible en Google Colab y 100% consistente con los contenidos del curso.


## Preguntas e hipótesis

**P1.** ¿Canciones con mayor *energy* tienden a ser más populares?  
**H1.** La relación entre *energy* y *popularidad* es **positiva**.

**P2.** ¿La *danceability* se asocia con la popularidad?  
**H2.** La relación entre *danceability* y *popularidad* es **positiva**.

**P3.** ¿El *tempo/BPM* y la *duración* influyen en la popularidad?  
**H3.** Tempos medios y duraciones moderadas tenderían a puntajes de popularidad más altos.

**P4.** ¿El *valence* (color emocional) se refleja en la recepción del público?  
**H4.** *Valence* tiene una **asociación moderada** con la popularidad.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")


In [ ]:
url = "https://raw.githubusercontent.com/gerardohamburg/CoderHouse/main/songs_normalize.csv"
df = pd.read_csv(url)
df.columns = (df.columns
              .str.strip()
              .str.lower()
              .str.replace(" ", "_"))

df.shape, df.dtypes.head()


### Exploración inicial y valores ausentes

In [ ]:
na = df.isna().sum().to_frame("faltantes")
na.T


### Resúmenes numéricos

In [ ]:
desc = df.select_dtypes(include="number").describe().T
desc


### Selección de variables

In [ ]:
def pick(cols):
    for c in cols:
        if c in df.columns:
            return c

col_pop = pick(["popularity","target","streams"])
col_energy = pick(["energy"])
col_dance = pick(["danceability"])
col_valence = pick(["valence"])
col_tempo = pick(["tempo","bpm"])
col_loud = pick(["loudness","loudness_db"])
col_dur = pick(["duration_ms","duration","duration_s"])
col_year = pick(["released_year","year","release_year","release_years","year_released"])

sel_nums = [c for c in [col_pop,col_energy,col_dance,col_valence,col_tempo,col_loud,col_dur] if c]
df[sel_nums].head()


## Visualizaciones univariadas

In [ ]:
targets = [col_pop, col_energy, col_dance]
targets = [c for c in targets if c]
n = len(targets)
fig, axs = plt.subplots(1, n, figsize=(5*n, 4))
if n == 1:
    axs = [axs]
for ax, c in zip(axs, targets):
    sns.histplot(df[c], kde=True, ax=ax)
    ax.set_title(c)
plt.tight_layout()
plt.show()


## Boxplots por grupos (año binned)

In [ ]:
if col_year and col_pop:
    dec = (df[col_year]//10)*10
    df_plot = df.assign(decade=dec)
    fig, axs = plt.subplots(1, min(3, len([x for x in [col_pop,col_energy,col_dance,col_valence] if x])), figsize=(14,4))
    cols_to_plot = [x for x in [col_pop,col_energy,col_dance,col_valence] if x][:len(axs)]
    for ax, c in zip(axs, cols_to_plot):
        sns.boxplot(data=df_plot, x="decade", y=c, ax=ax)
        ax.set_xlabel("decade")
        ax.set_title(f"{c} por década")
    plt.tight_layout()
    plt.show()


## Visualizaciones bivariadas y multivariadas

In [ ]:
x = col_energy if col_energy else sel_nums[0]
y = col_dance if col_dance else sel_nums[1] if len(sel_nums)>1 else sel_nums[0]
h = None
if col_pop:
    try:
        h = pd.qcut(df[col_pop], q=4, labels=["Q1","Q2","Q3","Q4"])
    except Exception:
        h = df[col_pop]
s = col_tempo if col_tempo else None

plt.figure(figsize=(7,5))
sns.scatterplot(x=df[x], y=df[y], hue=h, size=df[s] if s else None, sizes=(20,140), alpha=0.8)
plt.xlabel(x); plt.ylabel(y)
plt.title("Dispersión multivariada")
plt.tight_layout()
plt.show()


## Mapa de correlaciones

In [ ]:
corr = df.select_dtypes(include="number").corr(numeric_only=True)
plt.figure(figsize=(10,7))
sns.heatmap(corr, cmap="RdBu_r", center=0)
plt.title("Correlaciones")
plt.tight_layout()
plt.show()

corr[col_pop].sort_values(ascending=False).head(10) if col_pop in corr.columns else corr.iloc[0:0]


## Detección visual de outliers (IQR)

In [ ]:
def iqr_bounds(s):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    return q1 - 1.5*iqr, q3 + 1.5*iqr

cols_check = [c for c in [col_energy,col_dance,col_tempo,col_loud,col_dur,col_valence,col_pop] if c]
out_counts = {}
for c in cols_check:
    lo, hi = iqr_bounds(df[c].dropna())
    out_counts[c] = int(((df[c] < lo) | (df[c] > hi)).sum())
pd.Series(out_counts, name="outliers_IQR").sort_values(ascending=False)


## Conclusiones

1. No se observan problemas de datos ausentes relevantes en las variables numéricas principales del conjunto.  
2. Las distribuciones univariadas muestran la escala y la dispersión típica de métricas de audio como **energy** y **danceability**; la **popularidad**, cuando está disponible, presenta asimetrías esperables por concentración de éxitos.  
3. Los boxplots por década sugieren que algunas métricas varían en el tiempo; esto ayuda a contextualizar comparaciones y a evitar confundir efectos de época con asociaciones entre variables.  
4. La dispersión multivariada apoya las hipótesis **H1** y **H2** de asociación positiva entre **energy**, **danceability** y **popularidad**; la fuerza del patrón puede cambiar según el período o subgénero.  
5. El mapa de correlaciones ayuda a priorizar variables para modelado posterior; típicamente aparecen asociaciones entre pares como **energy–loudness** y **tempo–duration** más débiles.  
6. La detección IQR identifica observaciones extremas que conviene revisar antes de modelar para evitar sesgos o métricas inestables.

Estos resultados son coherentes con los contenidos del curso y dejan listo el terreno para una fase de modelado supervisado básico, siempre manteniendo la trazabilidad con las visualizaciones y resúmenes numéricos producidos aquí.
